In [25]:
#Setting up environment
import pandas as pd
import psycopg2
from IPython.display import display
from sqlalchemy import create_engine

In [26]:
#Importing DATASET and show your first lines
df = pd.read_csv('Chocolate Sales.csv', sep=',')
display(df.head())

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,04-Jan-22,"$5,320",180
1,Van Tuxwell,India,85% Dark Bars,01-Aug-22,"$7,896",94
2,Gigi Bohling,India,Peanut Butter Cubes,07-Jul-22,"$4,501",91
3,Jan Morforth,Australia,Peanut Butter Cubes,27-Apr-22,"$12,726",342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,24-Feb-22,"$13,685",184


In [27]:
#LAST LINES OF DATAFRAME
df.tail(5)

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
1089,Karlen McCaffrey,Australia,Spicy Special Slims,17-May-22,"$4,410",323
1090,Jehu Rudeforth,USA,White Choc,07-Jun-22,"$6,559",119
1091,Ches Bonnell,Canada,Organic Choco Syrup,26-Jul-22,$574,217
1092,Dotty Strutley,India,Eclairs,28-Jul-22,"$2,086",384
1093,Karlen McCaffrey,India,70% Dark Bites,23-May-22,"$5,075",344


In [28]:
display(df.shape)
display(df.isnull().sum())
if df.isnull().sum().sum() == 0:
    print('Nenhum valor nulo nas colunas')

(1094, 6)

Sales Person     0
Country          0
Product          0
Date             0
Amount           0
Boxes Shipped    0
dtype: int64

Nenhum valor nulo nas colunas


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Sales Person   1094 non-null   object
 1   Country        1094 non-null   object
 2   Product        1094 non-null   object
 3   Date           1094 non-null   object
 4   Amount         1094 non-null   object
 5   Boxes Shipped  1094 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 51.4+ KB


In [30]:
#Transform ''date'' to datetime
df['Date'] = pd.to_datetime(df['Date'],  format='%Y-%m-%d')
#FORMATING 'AMOUNT' to INT:
df['Amount'] = df['Amount'].replace({'$':'', ',' : ''}, regex=True).astype(int)

ValueError: time data "04-Jan-22" doesn't match format "%Y-%m-%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
display(df.head())

,Sales Person,Country,Product,Date,Amount,Boxes Shipped
0,Jehu Rudeforth,UK,Mint Chip Choco,2022-01-04,5320,180
1,Van Tuxwell,India,85% Dark Bars,2022-08-01,7896,94
2,Gigi Bohling,India,Peanut Butter Cubes,2022-07-07,4501,91
3,Jan Morforth,Australia,Peanut Butter Cubes,2022-04-27,12726,342
4,Jehu Rudeforth,UK,Peanut Butter Cubes,2022-02-24,13685,184


In [ ]:
#creating metrics

#PRODUCT WITH THE MOST REVENUE:
product_most_sales = df.groupby('Product')['Amount'].sum().reset_index(name='Total_Sales').sort_values(by='Total_Sales', ascending=False)
display(product_most_sales)

,Product,Total_Sales
19,Smooth Sliky Salty,349692
0,50% Dark Bites,341712
21,White Choc,329147
17,Peanut Butter Cubes,324842
10,Eclairs,312445
3,99% Dark & Pure,299796
2,85% Dark Bars,299229
16,Organic Choco Syrup,294700
20,Spicy Special Slims,293454
14,Mint Chip Choco,283969


In [ ]:
#THE MOST Boxes Shipped:>
most_shipped = df.groupby('Product')['Boxes Shipped'].max().reset_index(name='Boxes Shipped').sort_values(by='Boxes Shipped', ascending=False)
display(most_shipped)

,Product,Boxes Shipped
9,Drinking Coco,709
0,50% Dark Bites,708
7,Caramel Stuffed Bars,614
1,70% Dark Bites,597
10,Eclairs,591
12,Manuka Honey Choco,581
19,Smooth Sliky Salty,554
6,Baker's Choco Chips,524
20,Spicy Special Slims,520
4,After Nines,520


In [ ]:
#REGION WITH THE MOST ORDERS
best_region = df.groupby('Country')['Country'].count().reset_index(name='Quantity Orders')
display(best_region)

,Country,Quantity Orders
0,Australia,205
1,Canada,175
2,India,184
3,New Zealand,173
4,UK,178
5,USA,179


In [ ]:
#REGIONS WITH MORE REVENUES:
revenues_country = df.groupby('Country')['Amount'].sum().reset_index(name='Revenue').sort_values(by='Revenue',ascending=False)
display(revenues_country)

,Country,Revenue
2,India,"$7,896 $4,501 $5,376 $168 $2,443 $6,307 $4,382..."
4,UK,"$5,320 $13,685 $13,685 $4,704 $1,085 $3,003 $1..."
3,New Zealand,"$3,990 $168 $8,379 $9,492 $5,061 $1,722 $147 $..."
5,USA,"$3,703 $3,017 $1,267 $4,753 $4,025 $9,583 $938..."
0,Australia,"$12,726 $3,080 $2,835 $6,790 $6,888 $7,672 $4,..."
1,Canada,"$1,442 $4,067 $8,799 $1,652 $2,163 $4,781 $7,6..."


In [33]:
#CONEXAO COM BANCO DE DADOS:
dbname = '02_etl_python'
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'

conn = psycopg2.connect(dbname = dbname,
                        user = user,
                        password = password,
                        host = host,
                        port = port)

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{dbname}")

In [35]:
#CREATING TABLES WITH NEW METRICS
best_region.to_sql('best_region', engine, if_exists='replace', index=False)
best_region.to_csv('vendas_processadas', index=False)